In [1]:
import sys,os
[sys.path.append(x[0]) for x in os.walk('SL2P_PYTHON_package')]
import dictionariesSL2P

from matplotlib import pyplot as plt
import os,numpy
import rasterio
import os
import pandas as pd
import pickle
import tqdm

In [2]:
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

In [3]:
def readfile(fn):
    with open(fn, "rb") as fp:   #Pickling
            sampleDictionary = pickle.load(fp)
    return sampleDictionary

In [4]:
def writefile(sampleDictionary,fn):
    with open(fn, "wb") as fp:   #Pickling
        pickle.dump(sampleDictionary, fp) 
    fp.close()
    return 

In [5]:
def runSL2P(SRsamples,variableName,imageCollectionName,sl2p_inputs_bands):
    blockPrint()
    keys=list(SRsamples.keys())
    for key in  keys:
        for esu in range(len(SRsamples[key])):
            att=list(SRsamples[key][esu].keys())[1]
            samples=SRsamples[key][esu][att]
            estimatesdict=SRsamples[key][esu]
            estimatesdict.pop(att) 
            if not samples.empty:
                if imageCollectionName.startswith('S2'):
                        samples=samples.rename(columns={"B3": "B03","B4": "B04","B5": "B05","B6": "B06","B7": "B07"})
                estimates=SL2P(samples,variableName,imageCollectionName,sl2p_inputs_bands)
                estimates=estimates.drop(['QC_output'],axis=1).rename(columns={'QC_input':'QC'})
                estimates=pd.concat([samples[['date', 'longitude', 'latitude']],estimates],axis=1)
            else:
                estimates=pd.DataFrame()
                
            estimatesdict[SL2P.__name__]=estimates 
                       
            SRsamples[key][esu] =estimatesdict
                #SRsamples[key][esu][SL2P.__name__]=SRsamples[key][esu].pop(att) # rename dictionary attribute 
    return SRsamples      

### Select SL2P version (SL2PV0 or SL2PCCRS)

In [6]:
fn='testdata/NEON_2013-22_RMs_updated_CCRSInSitu2019_COPERNICUS_S2_SR_HARMONIZED_Surface_reflectance_testdata.pkl'

In [19]:
algorithm='SL2PCCRS'                   # 'SL2PV0' or 'SL2PCCRS'
imageCollectionName="S2_L2A"           # 'S2_L2A', 'S2_L2A_10m', 'LC08_T1L2', 'LC09_T1L2', 'HLSL30'
variableName='fAPAR'                     # 'Albedo', 'fAPAR', 'fCOVER', 'LAI', 'CCC', 'CWC'

In [20]:
#### Input data Control ***-
if variableName not in dictionariesSL2P.make_net_options().keys():
    raise ValueError(('Supported vegetation valiables: %s'%(dictionariesSL2P.make_net_options().keys())))
if imageCollectionName not in dictionariesSL2P.make_net_options()[variableName].keys():
    raise ValueError(('Supported EO datasets: %s'%(dictionariesSL2P.make_net_options()[variableName].keys())))  
       
sl2p_inputs_bands=dictionariesSL2P.make_net_options()[variableName][imageCollectionName]['inputBands']  
print ('SL2P input data format for %s data: %s' %(imageCollectionName,sl2p_inputs_bands))

In [21]:
if algorithm=='SL2PV0':
    from SL2P import SL2P as SL2P
elif algorithm=='SL2PCCRS':
    from SL2P import SL2PCCRS as SL2P
else:
    raise ValueError(('SL2P version: SL2PV0 and SL2PCCRS')) 

##### 1- Read SR data (.pkl)

In [22]:
SRsamples=readfile(fn)

##### 2- Run SL2P

In [23]:
BIOsamples=runSL2P(SRsamples,variableName,imageCollectionName,sl2p_inputs_bands)

##### 3- Export product (.pkl)

In [24]:
ofn=fn.replace('.pkl','_%s_%s.pkl'%(SL2P.__name__,variableName))
writefile(BIOsamples,ofn)